# Outline

- Who needs a priority queue? Emergency departments, data compression algorithms, etc.
- How they work? By organizing data so that the highest (or lowest) valued element is the next one to return.

## Example

Waiting room of an ER, in order of arrivals:

`[broken angle, stroke, sprained wrist, kidney stone]`

Reorganized as a priority queue:

`[stroke, broken angle, sprained wrist, kidney stone]`

After the first case (`stroke`) is taken, the list will be reorganized with the most severe case moved to the front.

`[kidney stone, broken angle, sprained wrist]`

Notice that the list is _not_ sorted; just the most important item, based on some metric, is placed at the front. The only requirement here is that `list[0]` is always the most important item. The remaining elements can be randomly organized as far as we are concerned.


## Naive priority queue

- Linear scan to find the most importnat element and bring it to the front.
- Implement with integers before writing a generic class.
- Intro to generics


In [ ]:
class SimplePriorityQ:

    _DEFAULT_CAPACITY: int = 10

    def __init__(self, capacity: int = _DEFAULT_CAPACITY) -> None:
        """Initialize an empty priority queue with given capacity."""
        self._underlying: list[int] = []
        self._capacity: int = capacity
        self._size: int = 0

    def insert(self, value: int) -> None:
        """Insert value into the priority queue."""
        if self._size >= self._capacity:
            raise Exception("Priority queue is full")
        self._underlying.append(value)
        self._size += 1
        self._move_important_to_front()

    def _move_important_to_front(self):
        max_idx = 0
        for i in range(1, len(self._underlying)):
            if self._underlying[i] > self._underlying[max_idx]:
                max_idx = i
        # swap
        temp = self._underlying[0]
        self._underlying[0] = self._underlying[max_idx]
        self._underlying[max_idx] = temp

    def remove_max(self) -> int:
        """Remove and return the maximum element from the priority queue."""
        if self.is_empty():
            raise Exception("Priority queue is empty")
        most_important = self._underlying[0]
        self._underlying[0] = self._underlying[len(self._underlying) - 1]
        temp_list = [None] * (len(self._underlying) - 1)
        for i in range(1, len(self._underlying)):
            temp_list[i - 1] = self._underlying[i]
        self._underlying = temp_list
        self._move_important_to_front
        self._size -= 1
        return most_important

    def is_empty(self) -> bool:
        """Return True if the priority queue is empty, False otherwise."""
        return len(self._underlying) == 0